# Task 3
## Creating alternative differential expression modules
---
## Relevance:  
While tools like sceptre are very accurated calculating differential perturbation it can take a long time to calculate the differential expression for all genes. 
Another researchers can have interest in extend the differential expression analysis to tools. The aim of this task is to create a module that can be used to calculate differential expression using a mudata contaning  a mudata object with the informations about guides and scRNA-seq , covariates and coordinate information 

## Practical description: TODO

-  [x] Download the mudata_results.h4mu and transform in a mudata_processed_guides_and_scrna.h5mu by removing unecessary information (code already provided "Downloading the File and creating mudata with guides and scRNA")
-  [ ] Create the enviroment by running the provided code. Check on  "Creating the enviroment" in the last notebook session
-  [ ] Create a nextflow process called differential_pertubation that takes as input the mudata_processed_guides_and_scrna.h5mu and a parameter contaning the information about in_trans=TRUE or FALSE.
-  [ ] Use the script  differential_pertubation.py to calculate the differential expression.
- [ ] return all the result in a dir called  results_dir
- [ ] Consider if we need a single file for each diff perturbation tool or if we can receive the current mudata and  

# Downloading the File and creating mudata with guides and scRNA

In [1]:

!wget https://www.dropbox.com/s/5jysq76bttem3jd/mudata_results.h5mu?dl=1 -o mudata_results.h5mu
!mv mudata_results.h5mu?dl=1 mudata_results.h5mu


In [6]:
import mudata as md
mu_results = md.read('/home/jovyan/mudata_results.h5mu')
md.MuData({'guides': mu_results['guides'], 'scRNA':mu_results['scRNA']}).write('mudata_processed_guides_and_scrna.h5mu')


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/conda/lib/python3.10/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


## Developing

In [68]:
!conda env list

# conda environments:
#
hon_scanpy               /home/jovyan/.conda/envs/hon_scanpy
perturbvi-conda          /home/jovyan/.conda/envs/perturbvi-conda
perturbvi-conda-jamboree     /home/jovyan/.conda/envs/perturbvi-conda-jamboree
base                     /opt/conda
pygenomictracks          /opt/conda/envs/pygenomictracks



In [69]:
%%writefile differential_pertubation.nf


params.IN_TRANS = 'TRUE'
params.MU_GUIDE_AND_SCRNA = '/home/jovyan/mudata_processed_guides_and_scrna.h5mu'



useMamba = true
//place to cash conda/mamba packages
cacheDir =  '/home/jovyan/' 

process differential_pertubation {

   debug true
   conda '/home/jovyan/.conda/envs/perturbvi-conda-jamboree'
    
   input:
   path (mudata_guides_rna)
   val (in_trans)
   output:
   path 'results_dir', emit: differential_results_dir
   script:
    """
     mkdir results_dir
     chmod 700 /home/jovyan/differential_pertubation.py
     /home/jovyan/differential_pertubation.py ${mudata_guides_rna} ${in_trans}
     
    """
    
}

workflow {
    differential_pertubation(params.MU_GUIDE_AND_SCRNA,
                             params.IN_TRANS )
    
}


Overwriting differential_pertubation.nf


In [70]:
%%writefile differential_pertubation.py
#!/usr/bin/env python

import sys
import os
import mudata as md
import perturbvi


mu_guides_and_rna = md.read(sys.argv[1])
in_trans = sys.argv[2]



#save all the results (or a mudata witht the results) in the results_dir dir. 
#Create a  new dir if needed 


Overwriting differential_pertubation.py


In [72]:
!nextflow differential_pertubation.nf -w diff_perturb_test -with-conda

N E X T F L O W  ~  version 22.10.6
Launching `differential_pertubation.nf` [agitated_plateau] DSL2 - revision: 0f1943b7fd
[-        ] process > differential_pertubation -

executor >  local (1)
[92/85e43d] process > differential_pertubation [  0%] 0 of 1

executor >  local (1)
[92/85e43d] process > differential_pertubation [100%] 1 of 1 ✔




## Sandbox instructions example

obs:
- This results were created only for a few guides
- The small amount of cells reduced the significance, even these positive promoter genes aren't perfect

In [21]:
import mudata as md

mu_guides_and_rna = md.read('/home/jovyan/mudata_processed_guides_and_scrna.h5mu')

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [22]:
mu_guides_and_rna['guides']

AnnData object with n_obs × n_vars = 5109 × 98
    obs: 'bath_number', 'percent_mito', 'log_number_of_detected_genes', 'log_total_gene_count', 'log_total_guide_count'
    var: 'feature_name', 'guide_chr', 'guide_end', 'guide_start', 'guide_number', 'target_elements'

In [23]:
mu_guides_and_rna['guides'].obs

,bath_number,percent_mito,log_number_of_detected_genes,log_total_gene_count,log_total_guide_count
AAACCTGAGAGGGCTT,1,0.000000,5.513429,5.517453,0.000000
AAACCTGCAAATACAG,1,0.022816,8.161660,8.161946,0.000000
AAACCTGCACTCGACG,1,0.051611,7.784473,7.784889,0.000000
AAACCTGCAGTCTTCC,1,0.053218,7.934155,7.934513,0.693147
AAACCTGCATGCCTAA,1,0.045982,8.531490,8.531688,0.000000
...,...,...,...,...,...
TTTGGTTTCCTGCAGG,1,0.027573,7.791110,7.791523,2.079442
TTTGGTTTCGTCCGTT,1,0.027464,8.423102,8.423322,0.693147
TTTGTCAAGCGAGAAA,1,0.040875,7.649693,7.650169,0.000000
TTTGTCACACATAACC,1,0.031246,8.506739,8.506941,1.386294


In [24]:
mu_guides_and_rna['guides'].var

,feature_name,guide_chr,guide_end,guide_start,guide_number,target_elements
feature_id,,,,,,
BROX|1_sgrna_chr1:222712600:222712620,BROX|1_sgrna_chr1:222712600:222712620,chr1,222712620,222712600,1,BROX
BROX|2_sgrna_chr1:222712744:222712764,BROX|2_sgrna_chr1:222712744:222712764,chr1,222712764,222712744,2,BROX
CCND2|1_sgrna_chr12:4273763:4273783,CCND2|1_sgrna_chr12:4273763:4273783,chr12,4273783,4273763,1,CCND2
CCND2|2_sgrna_chr12:4273832:4273852,CCND2|2_sgrna_chr12:4273832:4273852,chr12,4273852,4273832,2,CCND2
CEBPB|1_sgrna_chr20:50190830:50190850,CEBPB|1_sgrna_chr20:50190830:50190850,chr20,50190850,50190830,1,CEBPB
...,...,...,...,...,...,...
UAP1|2_sgrna_chr1:162561996:162562016,UAP1|2_sgrna_chr1:162561996:162562016,chr1,162562016,162561996,2,UAP1
UBE2T|1_sgrna_chr1:202341918:202341938,UBE2T|1_sgrna_chr1:202341918:202341938,chr1,202341938,202341918,1,UBE2T
UBE2T|2_sgrna_chr1:202341907:202341927,UBE2T|2_sgrna_chr1:202341907:202341927,chr1,202341927,202341907,2,UBE2T


In [25]:
mu_guides_and_rna['scRNA']

AnnData object with n_obs × n_vars = 5109 × 16030
    obs: 'bath_number', 'percent_mito', 'log_number_of_detected_genes', 'log_total_gene_count', 'log_total_guide_count'
    var: 'feature_name', 'n_cells', 'transcript_chr', 'transcript_start', 'transcript_end', 'ensg'

In [26]:
mu_guides_and_rna['scRNA'].var

,feature_name,n_cells,transcript_chr,transcript_start,transcript_end,ensg
feature_name,,,,,,
AL627309.1,AL627309.1,79,NOT_FOUND,0,0,ENSG00000238009
AL627309.5,AL627309.5,417,NOT_FOUND,0,0,ENSG00000241860
AP006222.1,AP006222.1,240,NOT_FOUND,0,0,ENSG00000228463
AP006222.2,AP006222.2,51,NOT_FOUND,0,0,ENSG00000286448
AL732372.2,AL732372.2,90,NOT_FOUND,0,0,ENSG00000237094
...,...,...,...,...,...,...
AL592183.1,AL592183.1,250,NOT_FOUND,0,0,ENSG00000273748
AC007325.4,AC007325.4,58,NOT_FOUND,0,0,ENSG00000278817
AL354822.1,AL354822.1,525,NOT_FOUND,0,0,ENSG00000278384


In [27]:
mu_guides_and_rna['scRNA'].obs

,bath_number,percent_mito,log_number_of_detected_genes,log_total_gene_count,log_total_guide_count
AAACCTGAGAGGGCTT,1,0.000000,5.513429,5.517453,0.000000
AAACCTGCAAATACAG,1,0.022816,8.161660,8.161946,0.000000
AAACCTGCACTCGACG,1,0.051611,7.784473,7.784889,0.000000
AAACCTGCAGTCTTCC,1,0.053218,7.934155,7.934513,0.693147
AAACCTGCATGCCTAA,1,0.045982,8.531490,8.531688,0.000000
...,...,...,...,...,...
TTTGGTTTCCTGCAGG,1,0.027573,7.791110,7.791523,2.079442
TTTGGTTTCGTCCGTT,1,0.027464,8.423102,8.423322,0.693147
TTTGTCAAGCGAGAAA,1,0.040875,7.649693,7.650169,0.000000
TTTGTCACACATAACC,1,0.031246,8.506739,8.506941,1.386294


# Creating the enviroment

In [63]:
%%writefile pertubvi.yaml
name: perturbvi-conda-jamboree
channels:
    - conda-forge
    - defaults
dependencies:
    - anndata
    - scvi-tools>=0.20.1
    - pytorch
    - pyro-ppl
    - mudata
    - pandas >= 1.5.3
    - session-info
    - pip
    - pip:
        - "https://test-files.pythonhosted.org/packages/de/9b/b9aeb1c18b7604bfc3873b0d2314018d99f37921f5f64023114a0a286582/perturbvi-0.0.1-py3-none-any.whl"

Overwriting pertubvi.yaml


In [64]:
!mamba env create -f pertubvi.yaml --force

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
[+] 0.1s
pkgs/main/linux-64 ━━╸━━━━━━━━━━━━━━━╸━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/main/noarch   ━━╸━━━━━━━━━━━━━━━╸━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/linux-64    ━━━━━━━━━━╸━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/noarch      ━━━━━━━━━━╸━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s[+] 0.2s
pkgs/main/linux-64 ━━━╸━━━━━━━━━━━━━━━╸━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.2s
pkgs/main/noarch   ━━━╸━━━━━━━━━━━━━━━╸━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.2s
pkgs/r/linux-64    ━━━━━━━━━━━╸━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.2s
pkgs/r/noarch      ━━━━━━━━━━━╸━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.2spkgs/main/linux-64                                            No change
pkgs/main/noarch                                              No change
pkgs/r/linux-64                        

In [ ]:
!source activate  perturbvi-conda; python -m 